<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning un model con Keras**

---


Un cop fet tot el treball de preprocessament de dades, només queden uns quants passos per entrenar el model. Cal tenir en compte, però, que l'ordre model.fit() s'executarà molt lentament en una CPU. Si no teniu una GPU configurada, podeu accedir a GPU o TPU gratuïtes a Google Colab.

In [1]:
!pip install tf-keras --quiet


In [2]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"


In [ ]:
!pip install datasets


from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

# **Procés:**

---
1. **Importació de llibreries**.

  *  load_dataset: per carregar conjunts de dades de Hugging Face.

  * AutoTokenizer: crea el tokenitzador compatible amb el model BERT.

  * DataCollatorWithPadding: per agrupar exemples amb padding automàtic.

1. **Es carrega la tasca MRPC** (Microsoft Research Paraphrase Corpus) del benchmark GLUE, que conté parelles de frases i una etiqueta que indica si són parafrasis (1) o no (0).
1. **Inicialitzar el tokenitzador de BERT**
  * Es fa servir el model preentrenat bert-base-uncased.

  * AutoTokenizer s’encarrega de tokenitzar les frases segons els requeriments de BERT.

1. **Funció de tokenització:**
  * Aquesta funció agafa cada parella de frases (sentence1 i sentence2) i les tokenitza.

  * Truncation=True talla les frases massa llargues perquè encaixin dins el màxim del model (512 tokens per a BERT).

1. **Aplicar la tokenització a tot el conjunt**:

  * map aplica la funció de tokenització a tots els exemples del conjunt (train, validation...).

  * batched=True vol dir que s’envien lots d’exemples per accelerar el procés.

1. **Collator per gestionar el padding automàtic, train i validation dataset**.

  * Converteix el conjunt de dades d'entrenament a un tf.data.Dataset.

  * Selecciona les columnes necessàries per a BERT: input_ids, attention_mask, token_type_ids.

  * label_cols=["labels"]: etiqueta binària per indicar si les frases són parafrasis.

  * shuffle=True: barreja les dades.

  * batch_size=8: mida de batch de 8 exemples.

1.




# **Entrenament**

---





In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

1. Importa el model BERT adaptat per classificació de frases
1. Carrega el model preentrenat i l’adapta a una classificació binària:
  * checkpoint és "bert-base-uncased", el mateix que vas utilitzar per tokenitzar.
  * from_pretrained(...) carrega els pesos del model entrenat per a una tasca general (com el MLM – Masked Language Modeling).
  * num_labels=2 especifica que la nova tasca és de classificació binària (dues etiquetes: paràfrasi o no paràfrasi).

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

1. **model.compile(...)**
  * optimizer="adam", fa servir l’optimitzador Adam, molt habitual en NLP, perquè s’adapta bé a gradients variables.

  * loss=SparseCategoricalCrossentropy(from_logits=True), Aquesta funció de pèrdua (loss function) s’utilitza quan tens etiquetes enteres (0 o 1), no en format one-hot.

  * from_logits=True indica que la sortida del model NO és una probabilitat (encara no s'ha aplicat softmax), així que la pèrdua internament l’aplicarà.

  * metrics=["accuracy"], Es mesura l’exactitud durant l'entrenament i la validació. És a dir, el % de respostes correctes.
1. **model.fit(...)**
  * tf_train_dataset: és el conjunt d'entrenament en format tf.data.Dataset, ja tokenitzat i amb padding.

  * validation_data=tf_validation_dataset: a cada època, avalua el model amb el conjunt de validació per veure si millora (o si hi ha overfitting).

  * Per defecte: Entrena durant 1 època (pots afegir epochs=3, per exemple). Mostra les mètriques a cada pas.

# **Millorant l'entrenament**

---

Defineix un optimitzador Adam amb learning rate variable, controlat per un planificador de decaïment (decay scheduler). Això pot millorar la convergència i estabilitat durant l'entrenament del model.

In [5]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [ ]:
import tensorflow as tf
import transformers
import importlib

print("TensorFlow:", tf.__version__)
print("Transformers:", transformers.__version__)

# Intentar detectar la versió de Keras instal·lada (ja sigui standalone o tf.keras)
try:
    import keras
    print("Keras (standalone):", keras.__version__)
except ImportError:
    print("Keras (standalone): No instal·lat")

try:
    print("Keras dins TensorFlow:", tf.keras.__version__)
except AttributeError:
    print("No s'ha pogut detectar tf.keras")


1. **Definir el número total de training steps:**
  * len(tf_train_dataset) és el nombre de batches (ja que és un tf.data.Dataset batxejat).
  * Es multiplica pel nombre d’èpoques (num_epochs) → això dona el nombre total de passos d’entrenament.
1. **Planificador de decaïment: PolynomialDecay**
  * Això crea una funció que:Comença amb un learning rate alt (aquí, 5e-5)

  * I el redueix progressivament fins a 0 al llarg dels num_train_steps

  * Fa servir una corba polinòmica per fer aquest canvi (més suau que un tall sec)

  * Resultat: el learning rate decreix lentament al llarg de l’entrenament → ajuda a fer passos grans al principi, però més petits i precisos al final.

1. **Optimitzador Adam amb learning rate variable**

  * Aquí es crea un optimitzador Adam, però no amb un valor fix de learning rate.

  * En comptes d’un valor com 5e-5, li passes el lr_scheduler, que anirà ajustant-lo automàticament a cada pas d’entrenament.

In [7]:

from tensorflow.keras.optimizers import Adam
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

In [ ]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

 **ULL a l'error** : La causa és que TensorFlow ha canviat a Keras 3 com a Keras "predeterminat" a partir de TF 2.16, i Keras 3 sovint s'instal·la juntament amb TF 2.15. Els errors d'aquest fil són perquè els objectes Keras 3 s'estan passant als objectes i codi del model Keras 2.

La solució més ràpida és instal·lar:



```
!pip tf-keras
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
```
Això farà que tf.keras apunti a Keras 2 i el codi hauria de funcionar com abans.